<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 4 / SPRINT 1 / NOTE 3*

---

# Document Classification

* 텍스트에서 특성을 추출하고 문서 분류기를 만들 수 있습니다
* 잠재의미분석(Latent Semantic Analysis,LSA)을 수행합니다
* Spacy 단어 임베딩을 사용합니다

### Warm up

다음 동영상을 시청하세요.
- [특이값 분해(SVD)의 기하학적 의미와 활용 소개](https://youtu.be/cq5qlYtnLoY)
    - 우리는 SVD를 통해 무엇을 얻고자 하는 것일까요?

다음 웹페이지를 읽어보세요. 이해가 안 되는 부분은 넘어가도 좋습니다.
- [Text classification](https://developers.google.com/machine-learning/guides/text-classification)
    - Introduction
    - Step 1: Gather Data
    - Step 2: Explore Your Data
    - Step 2.5: Choose a Model
    - Step 3: Prepare Your Data

---

여러분은 이미 머신러닝을 이용해 분류기를 학습시킬 수 있습니다. 그리고 텍스트 문서에서 어떻게 특성들을 추출하는지 배웠습니다. 이제 텍스트 문서를 분류하는 모델을 만들 차례 입니다!

## 텍스트에서 특성들을 추출하고 문서 분류기를 만들어 보겠습니다.

Sklearn 파이프라인을 사용하면 머신러닝 프로세스에 사용되는 여러 컴포넌트들을 쉽게 연결할 수 있었습니다.

파이프라인을 이용해 Raw 데이터 입력, 정제, 학습 프로세스를 하나의 함수에서 실행되는 것과 같이 편리하게 실행할 수 있습니다.

파이프라인을 사용하는 또 다른 이유는 하이퍼파라미터 튜닝을 쉽게 할 수 있기 때문입니다. 벡터화 과정중에 n-gram 범위라든지 최대 토큰의 수라든지 최적의 결과를 내기 위해 여러 파라미터들을 바꾸어 가며 실험을 해 보아야 합니다.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

20개 뉴스그룹으로 분류된 18,000개의 뉴스그룹 문서 데이터셋 입니다.
- [20newsgroups](https://scikit-learn.org/stable/datasets/index.html#the-20-newsgroups-text-dataset)
- 전자와 정치에 관한 두 개의 다른 카테고리 뉴스를 가져오겠습니다.


In [ ]:
categories = ['sci.electronics',
              'talk.politics.misc']

ng_train = fetch_20newsgroups(subset='train'
                             , remove=('headers', 'footers', 'quotes')
                             , categories=categories
                             )

ng_test = fetch_20newsgroups(subset='test'
                             , remove=('headers', 'footers', 'quotes')
                             , categories=categories
                             )

학습, 테스트 데이터가 분리되어 있습니다.

In [ ]:
len(ng_train.data), len(ng_test.data)

한 문서를 확인해 보겠습니다.

In [ ]:
ng_train.data[5]

이 문서의 타겟 레이블 입니다

In [ ]:
ng_train.target[5]

In [ ]:
ng_train.target_names

## 데이터를 살펴봅시다
- [Step 2: Explore Your Data](https://developers.google.com/machine-learning/guides/text-classification/step-2)

학습 모델을 만드는 일은 데이터 분석 과정 중 한 부분입니다. 모델링 전 데이터의 특성을 확인하고 이해하는 과정을 통해 더욱 좋은 모델을 만들 수 있게 됩니다. 데이터를 미리 잘 살펴보면 더 적은 데이터로 더 높은 성능을 가진 모델을 만들 수도 있습니다.

다음 웹페이지를 참고 하세요, https://developers.google.com/machine-learning/guides/text-classification/step-2
- explore_data.py 파일은 다음 URL에 있습니다. ipynb 폴더에 다운받아 import 하여 사용하세요
- [explore_data.py](https://github.com/google/eng-edu/blob/master/ml/guides/text_classification/explore_data.py)



In [ ]:
# 다음 웹페이지를 참고 하세요, https://developers.google.com/machine-learning/guides/text-classification/step-2
# explore_data.py 파일은 다음 URL에 있습니다. ipynb 폴더에 다운받아 import 하여 사용하세요
## https://github.com/google/eng-edu/blob/master/ml/guides/text_classification/explore_data.py
import explore_data as ed
import seaborn as sns
# sns.set()

In [ ]:
# Gets the median number of words per sample given corpus.
median_words_per_sample = ed.get_num_words_per_sample(ng_train.data)
print('Median words per sample: ', median_words_per_sample)

In [ ]:
ed.plot_sample_length_distribution(ng_train.data)

In [ ]:
ed.plot_class_distribution(ng_train.target)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [15, 4]

In [ ]:
# Plots the frequency distribution of n-grams.
# Arguments
#     samples_texts: list, sample texts.
#     ngram_range: tuple (min, mplt), The range of n-gram values to consider.
#         Min and mplt are the lower and upper bound values for the range.
#     num_ngrams: int, number of n-grams to plot.
#         Top `num_ngrams` frequent n-grams will be plotted.
ed.plot_frequency_distribution_of_ngrams(ng_train.data
                                     , ngram_range=(1, 2)
                                     , num_ngrams=50)

## 모델은 어떻게 선택할까요? 
- [Step 2.5: Choose a Model](https://developers.google.com/machine-learning/guides/text-classification/step-2-5)

앞서 간단히 살펴본 데이터의 수치를 기반해서 2.5단계에서는 어떤 분류 모델을 사용할 것인지 선택을 해 보겠습니다.

다음 플로우차트에서 어떻게 분류 모델을 선택해야 하는지 여러 실험을 통한 결과를 가지고 간략한 가이드를 제공합니다. 목표는 주어진 데이터세트에서 가능한 최선의 정확도를 낼 수 있고 동시에 학습시 계산량을 줄이는 것이었습니다. 최적의 방법을 찾기 위해 감성분석, 토픽 분류 등 여러 문제에 대해 12개 데이터 세트를 사용했으며 여러 모델구조를 사용해 45만번 이상의 실험을 수행하였습니다.

![flowchart](https://developers.google.com/machine-learning/guides/text-classification/images/TextClassificationFlowchart.png)

In [ ]:
# S/W ratio를 계산해 봅시다, 구글 flowchar에 따르면,
# S/W < 1500 일 경우 BoW 를 사용해 벡터화 하고 simple MLP 모델 or 앙상블 모델을 사용하는것을 추천하고 있습니다.
sw_ratio = len(ng_train.data) / median_words_per_sample
print('number of samples / median words per sample ratio: ', int(sw_ratio))

## TF-IDF 모델로 베이스라인을 만들어 봅시다

In [ ]:
# 파이프라인 구성 요소를 만듭니다
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [ ]:
import pandas as pd

# DTM을 생성합니다.
dtm = vect.fit_transform(ng_train.data)

dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())
dtm.shape

In [ ]:
# 파이프라인을 정의합니다
pipe = Pipeline([
    ('vect',vect)
    ,('clf', rfc)
])
pipe

In [ ]:
parameters = {
    'vect__max_df': (0.7, 1.0) # document frequency(%) 높을 경우 제거
    ,'vect__min_df': (2, 5, 10) # document frequency(횟수) 낮을 경우 제거
    ,'vect__max_features': (5000, 20000) # 코퍼스에서 term frequency 높은 순서대로 나열하여 제한
    ,'clf__n_estimators': (100, 500)
    ,'clf__max_depth': (10, 20, None)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(ng_train.data, ng_train.target)


In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_params_

In [ ]:
from sklearn.metrics import accuracy_score

# 테스트 데이터에 대해 정확도를 구해보겠습니다
y_test = grid_search.predict(ng_test.data)
accuracy_score(ng_test.target, y_test)

## 잠재의미분석(Latent Semantic Analysis, LSA)


잠재의미분석(LSA)는 BoW방법론을 사용해 만든 문서-단어행렬(DTM) 같은 행렬 데이터의 차원을 축소해 문서들에 숨어있는(latent) 의미(Topics)를 끌어내는 방법입니다.

이때 차원 축소에는 Truncated SVD(특이값 분해)를 사용해 원하는 문서나, 단어의 차원을 축소합니다.

물론 차원이 축소가 되더라도 기존에 문서나, 단어들 간의 거리관계는 어느정도 보존이 됩니다.


<img src="https://www.researchgate.net/profile/Konstantinos_Bougiatiotis/publication/321025221/figure/fig9/AS:668660309962763@1536432449448/Singular-value-decomposition-followed-by-rank-lowering-for-latent-semantic-indexing.jpg" alt="Singular value decomposition followed by rank lowering for latent semantic indexing"/>

SVD를 사용해 행렬 A를 $U, Σ, V^{T}$ 세 행렬의 곱으로 분해(decomposion) 합니다.
$$A=UΣV^{T}$$
$$AV=UΣ$$


$U$ 와 $V^T$ 의 열 벡터는 특이벡터(singular vector)라 불리는데
Truncated SVD는 특이값(singular value, $Σ$ 대각성분) 가운데 가장 큰 k개만 남기고 해당 특이값에 대응하는 특이벡터들로 원래 행렬 A를 근사하는 방법입니다.

물론 0보다 큰 특이값을 제거하면 정보의 손실이 발생하므로 적당히 필요한 차원만큼 k를 선택합니다.

여기서 만약 m개 문서, n개 단어로 이루어진 행렬을 truncated SVD로 분해해 다음과 같은 분해를 수행 했다면 다음과 같은 근사 식을 얻을 것이며

$$A_k=U_kΣ_kV^{T}_k$$

$U_k$와 $V_k$를 사용해 n차원으로 표현 되었던 문서를 k차원으로, 또는 m 차원으로 표현되었던 단어를 k 차원으로 표현할 수 있게 됩니다.



#### TruncatedSVD 를 사용하여 파이프라인에서 차원을 축소하고 분류문제를 풀어보겠습니다.

In [ ]:
import numpy as np
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV

# SVD를 사용한 차원 축소
from sklearn.decomposition import TruncatedSVD

vect = TfidfVectorizer(stop_words='english'
                       , ngram_range=(1,2)
                       , min_df=2
                       , max_df=0.7
                       , token_pattern=r'(?u)\b\w[A-Za-z]+\b' # 영문자만 사용
                       , max_features=10000
                      )

svd = TruncatedSVD(algorithm='randomized'
                   , n_iter=5
                   , random_state=2)

rfc = RandomForestClassifier(n_estimators=500, random_state=2)

In [ ]:
params = {
    # 100~500 사이의 정수 크기로 차원을 줄입니다
#     'svd__n_components': stats.randint(100, 500)
    'svd__n_components': stats.randint(2, 3) # 문서의 차원을 2로 고정
    
}

In [ ]:
# 1.Tfidf 문서 벡터화, 2. svd 차원축소, 3. 랜덤포레스트 분류기
pipe = Pipeline([
    ('vect', vect)
    , ('svd', svd)
    , ('clf', rfc)
])

In [ ]:
# Fit
random_search = RandomizedSearchCV(pipe,params, cv=3, n_iter=5, n_jobs=-1, verbose=1)
random_search.fit(ng_train.data, ng_train.target)

In [ ]:
# svd__n_components: Random search에서 선택된 줄어든 차원을 확인할 수 있습니다.
random_search.best_params_

In [ ]:
random_search.best_score_

In [ ]:
# 테스트셋으로 정확도를 계산합니다
y_test = random_search.predict(ng_test.data)
accuracy_score(ng_test.target, y_test)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(ng_test.target, y_test))

#### SVD를 따로 수행해서 행렬분해가 어떻게 되는지 확인해 보겠습니다.

- randomized_svd를 사용해서 $U$, $\Sigma$, $V^T$ 행렬을 구해보겠습니다.
- randomized_svd는 Truncated SVD에서 내부적으로 사용되는 기능입니다.

In [ ]:
# 학습 데이터를 TF-IDF vectorizer로 벡터화하여 사용하겠습니다.
A = random_search.best_estimator_.named_steps['vect'].transform(ng_train.data).todense()
X_test = random_search.best_estimator_.named_steps['vect'].transform(ng_test.data).todense()

In [ ]:
# DTM
A.shape, X_test.shape

In [ ]:
# randomized_svd를 사용하여 U, S(Sigma), VT(V transposed) 행렬을 얻습니다
from sklearn.utils.extmath import randomized_svd

U, S, VT = randomized_svd(A
                         , n_components=2 # 상위 특이값 2개를 선택합니다
                         , n_iter=5
                         , random_state=2)

In [ ]:
U.shape, S.shape, VT.shape

In [ ]:
U

In [ ]:
# 대각 성분(특이값)만 가져왔습니다.
S

In [ ]:
VT

$V_k^{T} V_k = I_k$

In [ ]:
VT @ VT.T

$A_k=U_kΣ_kV^{T}_k$ 

=> $A_k V_k = U_k\Sigma_k$

In [ ]:
# A는 A_k는 아니지만 결과는 같습니다.
AV = A @ VT.T
AV

In [ ]:
US = U @ np.diag(S)
US

테스트 문서들도 $V_k$를 사용해서 문서의 차원을 축소할 수 있습니다.

$XV_k$

In [ ]:
X_test_trans = X_test @ VT.T
X_test_trans

In [ ]:
X_test_trans.shape

#### truncatedSVD 결과물로 비교해 보겠습니다.

truncatedSVD 속성 `components_` 가 행렬 VT입니다.

In [ ]:
components = random_search.best_estimator_.named_steps['svd'].components_
print((components - VT).sum())

TruncatedSVD 속성을 조금 더 살펴봅시다.

먼저 특이값를 확인해 보겠습니다. 위에서 구한 S(Sigma)와 같습니다.

In [ ]:
print(random_search.best_estimator_.named_steps['svd'].singular_values_)
print(S)

차원이 줄어든 데이터(AV, US)의 분산값입니다.  

In [ ]:
print(random_search.best_estimator_.named_steps['svd'].explained_variance_)
print(np.var(US, axis=0))

이번에는 테스트 문서 샘플을 SVD를 사용해 차원 축소해 보겠습니다.

In [ ]:
# 예시로 테스트 문서 0 벡터를 사용합니다.
d0 = X_test[0]
print(d0.shape)
print(random_search.best_estimator_.named_steps['svd'].transform(d0))

테스트 데이터를 모두 변환해 보겠습니다.

In [ ]:
print(X_test.shape)
X_test_trans_2 = random_search.best_estimator_.named_steps['svd'].transform(X_test)
print(X_test_trans_2.shape)

위에서 직접 구한 값과 같습니다. (X_test @ VT.T)

In [ ]:
# 문서 0만 보겠습니다.
X_test_trans[0], X_test_trans_2[0]

LSA는 SVD를 통해 찾아진 topic들을 가지고 잠재적인 의미를 분석하는 것 입니다.

각 차원에 어떤 단어들이 모여 있는지 확인해 봅시다.
SVD를 통해 찾아진 두 잠재적 의미군(토픽)에 속하는 단어들이 들어간 것을 확인할 수 있습니다.

In [ ]:
# terms: 벡터화한 단어
terms = random_search.best_estimator_.named_steps['vect'].get_feature_names()
for index, topic in enumerate(components[:10]): # topic 최대 10개만 표시)
    print('Topic %d: '%(index + 1), [terms[i] for i in topic.argsort()[::-1][:6]]) # 수치가 큰 단어부터 최대 6단어 표시
    print('Score %d: '%(index + 1), [topic[i] for i in topic.argsort()[::-1][:6]]) # 

## Spacy 단어 임베딩을 사용합니다

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
doc = nlp("The tortoise jumped into the lake")

Spacy는 기본적으로 300차원으로 임베딩 합니다.

In [ ]:
len(doc.vector)

In [ ]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [ ]:
X_spacy = get_word_vectors(ng_train.data)

len(X_spacy) == len(ng_train.data)

In [ ]:
X_test_spacy = get_word_vectors(ng_test.data)

랜덤포레스트로 학습해 보겠습니다.

In [ ]:
rfc.fit(X_spacy, ng_train.target)

In [ ]:
y_test_spacy = rfc.predict(X_test_spacy)
accuracy_score(ng_test.target, y_test_spacy)

#### MLP(Multi-layer perceptron classifier)를 간단히 사용해보겠습니다

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs'
                   , alpha=1e-5
                   , hidden_layer_sizes=(16,2)
                   , random_state=2
                   )

In [ ]:
clf.fit(X_spacy, ng_train.target)

In [ ]:
y_test = clf.predict(X_test_spacy)
accuracy_score(ng_test.target, y_test_spacy)

## 참고자료

- [Singular Value Decomposition (the SVD)](https://youtu.be/mBcLRGuAFUk)
- [특이값 분해(Singular Value Decomposition, SVD)의 활용](https://darkpgmr.tistory.com/106)
- [numpy 벡터와 행렬연산 참고자료](https://ebbnflow.tistory.com/159)

